In [1]:
import numpy as np
from numpy.linalg import eigvals
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution
from qiskit.circuit import Parameter
from qiskit import transpile, QuantumCircuit

t = Parameter('t')

def trotter_step(ops, coe):
    """Returns a single trotter step consisting only of CNOT and single qubit gates.
        ops: list of Pauli strings
        coe: list of corresponding coefficients for each Pauli string
    """
    ham = PrimitiveOp(SparsePauliOp(ops, coe)) # Hamiltonian
    U = (t*ham).exp_i() # Unitary parameterized by t
    Ut = PauliTrotterEvolution().convert(U) # single Trotter step
    qc = transpile(Ut.to_circuit(), basis_gates=['u','cx']) # turn into circuit with only allowed gates
    return qc

def create_circuit(ops, coe, steps):
    """Returns a trotterized circuit
        ops: list of Pauli strings
        coe: list of corresponding coefficients for each Pauli string
        steps: number of trotter steps
    """
    ts = trotter_step(ops, coe).bind_parameters({t: 1/steps}) # trotter step
    nq = ts.width() # number of qubits
    qc = QuantumCircuit(nq)
    for i in range(steps):
        qc.append(ts, range(nq))
    return qc

def create_unitary(ops, coe):
    """Returns a unitary matrix
        ops: list of Pauli strings
        coe: list of corresponding coefficients for each Pauli string
    """
    return PrimitiveOp(SparsePauliOp(ops, coe)).exp_i().to_matrix()

def circuit_error(circuit, unitary):
    """Return error defined as abs(largest eigenvector) of Uc-U where
       Uc is the unitary representing the circuit and U=exp(-iH) the
       exact unitary matrix attempting to be recreated
        circuit: quantum circuit object (must be unitary, parameters must be bound)
        unitary: matrix
    """
    Uc = Operator(circuit).data
    return np.max(np.abs(eigvals(Uc - unitary)))
    

ops = ['IIX','XXI','ZZZ']
coe = [.5,.1,.3]
circ = create_circuit(ops, coe, 3)
#print(circ)
u = create_unitary(ops, coe)
print(circuit_error(circ,u))

0.04731750088337165


In [117]:
print(circ.decompose())

global phase: 5.8832
     ┌───────────────────────────────┐                                       »
q_0: ┤ U(0.333333333333333,-π/2,π/2) ├───────────────────────────────────────»
     └─────────┬────────────┬────────┘                                       »
q_1: ──────────┤ U(π/2,0,π) ├───────────■─────────────────────────────────■──»
               ├────────────┤         ┌─┴─┐┌───────────────────────────┐┌─┴─┐»
q_2: ──────────┤ U(π/2,0,π) ├─────────┤ X ├┤ U(0,0,0.0666666666666666) ├┤ X ├»
               └────────────┘         └───┘└───────────────────────────┘└───┘»
«                        ┌───┐┌────────────┐┌───┐»
«q_0: ───────────────────┤ X ├┤ U(0,0,0.2) ├┤ X ├»
«     ┌────────────┐┌───┐└─┬─┘└────────────┘└─┬─┘»
«q_1: ┤ U(π/2,0,π) ├┤ X ├──■──────────────────■──»
«     ├────────────┤└─┬─┘                        »
«q_2: ┤ U(π/2,0,π) ├──■──────────────────────────»
«     └────────────┘                             »
«     ┌───────────────────────────────┐                   »
«q_0: ┤ U

# Now with the actual Hamiltonian:

In [2]:
full_ham_ops = ["IIIIIIIIII", "IIIIIZIIII", "ZIIIIIIIII", "IIZIIIIIII", "IIIIIIIZII", "IIIIIIZIII", "IZIIIIIIII", 
                "IIIZIIIIII", "IIIIIIIIZI", "IIIIIIIIIZ", "IIIIZIIIII", "IIIIZIIIIZ", "ZIIIIIIIIZ", "IIIIZZIIII", 
                "ZIIIIZIIII", "IIIZIIIIZI", "IIIIIZIIIZ", "ZIIIZIIIII", "IIZIIIIZII", "IZIIIIZIII", "IIZIIIIIZI", 
                "IZIIIIIIZI", "IIIZIIIZII", "IIIZIIZIII", "IZIIIIIZII", "IIZIIIZIII", "IIZIIIIIIZ", "IZIIIIIIIZ", 
                "IIIIZIIZII", "IIIIZIZIII", "ZIIIIIIZII", "ZIIIIIZIII", "IIZIIZIIII", "IZIIIZIIII", "IIIIIIZZII", 
                "IZZIIIIIII", "IIIIIIIZIZ", "IIIIIIZIIZ", "IIZIZIIIII", "IZIIZIIIII", "IIIIIZIZII", "IIIIIZZIII", 
                "ZIZIIIIIII", "ZZIIIIIIII", "ZIIIIIIIZI", "IIIZIZIIII", "IIIIIIIZZI", "IIIIIIZIZI", "IIZZIIIIII", 
                "IZIZIIIIII", "IIIZIIIIIZ", "IIIIZIIIZI", "IIIIIZIIZI", "ZIIZIIIIII", "IIIIIIIIZZ", "IIIZZIIIII", 
                "XZZZXZIIII", "YZZZYZIIII", "ZIIIIXZZZX", "ZIIIIYZZZY", "XZZZXIIIIZ", "YZZZYIIIIZ", "IIIIZXZZZX", 
                "IIIIZYZZZY", "XZZZXXZZZX", "YZZZYXZZZX", "XZZZXYZZZY", "YZZZYYZZZY", "XZZXIIIIII", "YZZYIIIIII", 
                "IIIIIXZZXI", "IIIIIYZZYI", "XZZXIIIIIZ", "YZZYIIIIIZ", "IIIIZXZZXI", "IIIIZYZZYI", "IIIXXIIIIZ", 
                "IIIYYIIIIZ", "IIIIZIIIXX", "IIIIZIIIYY", "XZZXIZIIII", "YZZYIZIIII", "ZIIIIXZZXI", "ZIIIIYZZYI", 
                "IIIXXZIIII", "IIIYYZIIII", "ZIIIIIIIXX", "ZIIIIIIIYY", "IXZXIIXZXI", "IYZYIIXZXI", "IXZXIIYZYI", 
                "IYZYIIYZYI", "IIXXIIIXXI", "IIYYIIIXXI", "IIXXIIIYYI", "IIYYIIIYYI", "XZZXIIIIZI", "YZZYIIIIZI", 
                "IIIZIXZZXI", "IIIZIYZZYI", "IIIXXXZZZX", "IIIYYXZZZX", "IIIXXYZZZY", "IIIYYYZZZY", "XZZZXIIIXX", 
                "YZZZYIIIXX", "XZZZXIIIYY", "YZZZYIIIYY", "XZZZXXZZXI", "YZZZYXZZXI", "XZZZXYZZYI", "YZZZYYZZYI", 
                "XZZXIXZZZX", "YZZYIXZZZX", "XZZXIYZZZY", "YZZYIYZZZY", "XZZXIXZZXI", "YZZYIXZZXI", "XZZXIYZZYI", 
                "YZZYIYZZYI", "IXZZXIXZZX", "IYZZYIXZZX", "IXZZXIYZZY", "IYZZYIYZZY", "IIXZXIIXZX", "IIYZYIIXZX", 
                "IIXZXIIYZY", "IIYZYIIYZY", "XZZZXIIIII", "YZZZYIIIII", "IIIIIXZZZX", "IIIIIYZZZY", "IIIXXIIIII", 
                "IIIYYIIIII", "IIIIIIIIXX", "IIIIIIIIYY", "IXZZXXXIII", "IYZZYXXIII", "IXZZXYYIII", "IYZZYYYIII", 
                "IIXZXXZXII", "IIYZYXZXII", "IIXZXYZYII", "IIYZYYZYII", "XXIIIIXZZX", "YYIIIIXZZX", "XXIIIIYZZY", 
                "YYIIIIYZZY", "XZXIIIIXZX", "YZYIIIIXZX", "XZXIIIIYZY", "YZYIIIIYZY", "XXIIIXXIII", "YYIIIXXIII", 
                "XXIIIYYIII", "YYIIIYYIII", "XZXIIXZXII", "YZYIIXZXII", "XZXIIYZYII", "YZYIIYZYII", "IXZZXIXZXI", 
                "IYZZYIXZXI", "IXZZXIYZYI", "IYZZYIYZYI", "IIXZXIIXXI", "IIYZYIIXXI", "IIXZXIIYYI", "IIYZYIIYYI", 
                "IXZXIIXZZX", "IYZYIIXZZX", "IXZXIIYZZY", "IYZYIIYZZY", "IIXXIIIXZX", "IIYYIIIXZX", "IIXXIIIYZY", 
                "IIYYIIIYZY", "XZZIXIIIII", "YZZIYIIIII", "IIIIIXZZIX", "IIIIIYZZIY", "IIIIIXZZXZ", "IIIIIYZZYZ", 
                "XZZXZIIIII", "YZZYZIIIII", "IXXIIIXXII", "IYYIIIXXII", "IXXIIIYYII", "IYYIIIYYII", "XZIXIIIIII", 
                "YZIYIIIIII", "XIZXIIIIII", "YIZYIIIIII", "IIIIIXZIXI", "IIIIIYZIYI", "IIIIIXIZXI", "IIIIIYIZYI", 
                "IIZXXIIIII", "IIZYYIIIII", "IZIXXIIIII", "IZIYYIIIII", "IIIIIIIZXX", "IIIIIIIZYY", "IIIIIIZIXX", 
                "IIIIIIZIYY", "IXZXIXXIII", "IYZYIXXIII", "IXZXIYYIII", "IYZYIYYIII", "IIXXIXZXII", "IIYYIXZXII", 
                "IIXXIYZYII", "IIYYIYZYII", "XXIIIIXZXI", "YYIIIIXZXI", "XXIIIIYZYI", "YYIIIIYZYI", "XZXIIIIXXI", 
                "YZYIIIIXXI", "XZXIIIIYYI", "YZYIIIIYYI", "IIIXXIIIXX", "IIIYYIIIXX", "IIIXXIIIYY", "IIIYYIIIYY", 
                "ZIIXXIIIII", "ZIIYYIIIII", "IIIIIZIIXX", "IIIIIZIIYY", "XZZZXIIZII", "YZZZYIIZII", "XZZZXIZIII", 
                "YZZZYIZIII", "IIZIIXZZZX", "IIZIIYZZZY", "IZIIIXZZZX", "IZIIIYZZZY", "XZZZXIIIZI", "YZZZYIIIZI", 
                "IIIZIXZZZX", "IIIZIYZZZY", "XZIZXIIIII", "YZIZYIIIII", "XIZZXIIIII", "YIZZYIIIII", "IIIIIXZIZX", 
                "IIIIIYZIZY", "IIIIIXIZZX", "IIIIIYIZZY", "IIIXXXZZXI", "IIIYYXZZXI", "IIIXXYZZYI", "IIIYYYZZYI", 
                "XZZXIIIIXX", "YZZYIIIIXX", "XZZXIIIIYY", "YZZYIIIIYY", "IIIXXIIIZI", "IIIYYIIIZI", "IIIZIIIIXX", 
                "IIIZIIIIYY", "IIIXXIIZII", "IIIYYIIZII", "IIIXXIZIII", "IIIYYIZIII", "IIZIIIIIXX", "IIZIIIIIYY", 
                "IZIIIIIIXX", "IZIIIIIIYY", "XZZXIIIZII", "YZZYIIIZII", "XZZXIIZIII", "YZZYIIZIII", "IIZIIXZZXI", 
                "IIZIIYZZYI", "IZIIIXZZXI", "IZIIIYZZYI"]
full_ham_coe = [+1.0709274663656798,    -0.5772920990654372,    -0.5772920990654371,    -0.4244817531727134, 
                -0.42448175317271336,   -0.42448175317271336,   -0.4244817531727133,    -0.3899177647415215, 
                -0.38991776474152134,   -0.30101532158947975,   -0.30101532158947975,   +0.12357087224898464, 
                +0.11433954684977561,   +0.11433954684977561,   +0.11409163501020725,   +0.08470391802239534, 
                +0.08360121967246183,   +0.08360121967246183,   +0.0782363777898523,    +0.0782363777898523, 
                +0.07055432072184756,   +0.07055432072184756,   +0.07055432072184756,   +0.07055432072184756, 
                +0.06980180803300681,   +0.06980180803300681,   +0.06878552428444665,   +0.06878552428444665, 
                +0.06878552428444665,   +0.06878552428444665,   +0.06732342777645686,   +0.06732342777645686, 
                +0.06732342777645686,   +0.06732342777645686,   +0.06558452315458405,   +0.06558452315458405, 
                +0.06278876343471208,   +0.06278876343471208,   +0.06278876343471208,   +0.06278876343471208, 
                +0.06245512523136934,   +0.06245512523136934,   +0.06245512523136934,   +0.06245512523136934, 
                +0.060505605672770954,  +0.060505605672770954,  +0.06022550139954594,   +0.06022550139954594, 
                +0.06022550139954594,   +0.06022550139954594,   +0.05665606755281972,   +0.05665606755281972, 
                +0.053929860773588405,  +0.053929860773588405,  +0.053621410722614865,  +0.053621410722614865, 
                -0.0351167704024114,    -0.0351167704024114,    -0.0351167704024114,    -0.0351167704024114, 
                -0.03305872858775587,   -0.03305872858775587,   -0.03305872858775587,   -0.03305872858775587, 
                +0.0307383271773138,    +0.0307383271773138,    +0.0307383271773138,    +0.0307383271773138, 
                +0.02355744239583729,   +0.02355744239583729,   +0.023557442395837284,  +0.023557442395837284, 
                +0.0127339139792953,    +0.0127339139792953,    +0.0127339139792953,    +0.0127339139792953, 
                +0.011733623912074194,  +0.011733623912074194,  +0.011733623912074194,  +0.011733623912074194, 
                +0.010889407716094479,  +0.010889407716094479,  +0.010889407716094479,  +0.010889407716094479, 
                +0.010540187409026488,  +0.010540187409026488,  +0.010540187409026488,  +0.010540187409026488, 
                +0.010328819322301622,  +0.010328819322301622,  +0.010328819322301622,  +0.010328819322301622, 
                +0.010328819322301622,  +0.010328819322301622,  +0.010328819322301622,  +0.010328819322301622, 
                -0.00901204279263803,   -0.00901204279263803,   -0.00901204279263803,   -0.00901204279263803, 
                -0.008373361424264817,  -0.008373361424264817,  -0.008373361424264817,  -0.008373361424264817, 
                -0.008373361424264817,  -0.008373361424264817,  -0.008373361424264817,  -0.008373361424264817, 
                -0.0077644411821215335, -0.0077644411821215335, -0.0077644411821215335, -0.0077644411821215335, 
                -0.0077644411821215335, -0.0077644411821215335, -0.0077644411821215335, -0.0077644411821215335, 
                +0.006575744899182541,  +0.006575744899182541,  +0.006575744899182541,  +0.006575744899182541, 
                +0.005996760849734561,  +0.005996760849734561,  +0.005996760849734561,  +0.005996760849734561, 
                +0.005996760849734561,  +0.005996760849734561,  +0.005996760849734561,  +0.005996760849734561, 
                -0.005949019975734285,  -0.005949019975734285,  -0.005949019975734247,  -0.005949019975734247, 
                -0.004917569762418068,  -0.004917569762418068,  -0.00491756976241806,   -0.00491756976241806, 
                -0.004879740484191498,  -0.004879740484191498,  -0.004879740484191498,  -0.004879740484191498, 
                -0.004879740484191498,  -0.004879740484191498,  -0.004879740484191498,  -0.004879740484191498, 
                -0.004879740484191497,  -0.004879740484191497,  -0.004879740484191497,  -0.004879740484191497, 
                -0.004879740484191497,  -0.004879740484191497,  -0.004879740484191497,  -0.004879740484191497, 
                +0.004868302545087521,  +0.004868302545087521,  +0.004868302545087521,  +0.004868302545087521, 
                +0.004868302545087521,  +0.004868302545087521,  +0.004868302545087521,  +0.004868302545087521, 
                +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293, 
                +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293, 
                +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293, 
                +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293,  +0.004802531988356293, 
                +0.004479074568182561,  +0.004479074568182561,  +0.004479074568182561,  +0.004479074568182561, 
                +0.004360552555030484,  +0.004360552555030484,  +0.004360552555030484,  +0.004360552555030484, 
                +0.004217284878422759,  +0.004217284878422759,  +0.004217284878422759,  +0.004217284878422759, 
                -0.003818281201314288,  -0.003818281201314288,  -0.003818281201314288,  -0.003818281201314288, 
                -0.003818281201314288,  -0.003818281201314288,  -0.003818281201314288,  -0.003818281201314288, 
                +0.0036202487558837123, +0.0036202487558837123, +0.0036202487558837123, +0.0036202487558837123, 
                +0.0036202487558837123, +0.0036202487558837123, +0.0036202487558837123, +0.0036202487558837123, 
                -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337, 
                -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337, 
                -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337, 
                -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337,  -0.003466391848475337, 
                +0.003034656830204855,  +0.003034656830204855,  +0.003034656830204855,  +0.003034656830204855, 
                +0.0027757462269049522, +0.0027757462269049522, +0.0027757462269049522, +0.0027757462269049522, 
                +0.0027298828353264134, +0.0027298828353264134, +0.0027298828353264134, +0.0027298828353264134, 
                +0.0027298828353264134, +0.0027298828353264134, +0.0027298828353264134, +0.0027298828353264134, 
                +0.0023679368995844726, +0.0023679368995844726, +0.0023679368995844726, +0.0023679368995844726, 
                -0.0021498576488650843, -0.0021498576488650843, -0.0021498576488650843, -0.0021498576488650843, 
                -0.0021498576488650843, -0.0021498576488650843, -0.0021498576488650843, -0.0021498576488650843, 
                +0.00211113766859809,   +0.00211113766859809,   +0.00211113766859809,   +0.00211113766859809, 
                +0.00211113766859809,   +0.00211113766859809,   +0.00211113766859809,   +0.00211113766859809, 
                -0.00154067008970742,   -0.00154067008970742,   -0.00154067008970742,   -0.00154067008970742, 
                -0.0011822832324725804, -0.0011822832324725804, -0.0011822832324725804, -0.0011822832324725804, 
                -0.0011822832324725804, -0.0011822832324725804, -0.0011822832324725804, -0.0011822832324725804, 
                -0.0003518893528389501, -0.0003518893528389501, -0.0003518893528389501, -0.0003518893528389501, 
                -0.0003518893528389501, -0.0003518893528389501, -0.0003518893528389501, -0.0003518893528389501]
full_unitary = create_unitary(full_ham_ops, full_ham_coe)

In [118]:
%%time
trotter_steps = 10

circ = create_circuit(full_ham_ops, full_ham_coe, trotter_steps)
print('Circuit depth: ',circ.decompose().depth())
print(circuit_error(circ,full_unitary))

Circuit depth:  20393
1.0204880875326017
CPU times: user 8min 33s, sys: 447 ms, total: 8min 33s
Wall time: 8min 27s


In [119]:
%%time
trotter_steps = 100

circ = create_circuit(full_ham_ops, full_ham_coe, trotter_steps)
print('Circuit depth: ',circ.decompose().depth())
print(circuit_error(circ,full_unitary))

Circuit depth:  203903
1.0204799194405625
CPU times: user 1h 23min 48s, sys: 2.58 s, total: 1h 23min 51s
Wall time: 1h 23min 45s


In [5]:
%%time
trotter_steps = 10
circ = create_circuit(full_ham_ops, full_ham_coe, trotter_steps)
Uc = Operator(circ).data

KeyboardInterrupt: 